In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
[INST] Your role is an expert in the field of SMT Fuzzing and first order logic.\
Your task is to fuse two SMT problems into a single, logically consistent, and syntactically valid SMT problem.\
Think step by step. Only when you are done with all your steps, provide the answer based on all your intermediate steps.

## Semantic Fusion Methodology

Semantic Fusion is a methodology for validating SMT solvers by combining two equisatisfiable formulas into a fused formula that preserves satisfiability or unsatisfiability. The fusion process includes the following steps:

1. **Formula Concatenation:** 
    •	If the two formulas are **satisfiable (SAT Fusion)**, combine them using **conjunction** (logical AND).
	•	If the two formulas are **unsatisfiable (UNSAT Fusion)**, combine them using **disjunction** (logical OR).
2. **Variable Fusion:** 
    •	Introduce a **fresh variable** $z$ to fuse two variables $x$ and $y$ from the input formulas.
	•	Use a fusion function  \(f(x, y)\)  to define the relationship between  $x$ ,  $y$ , and  $z$ . Choose the fusion function based on the variable types (Int, Real, or Bool) using the table below:


3. **Variable Inversion:** Substitute some occurrences of $x$ and $y$ in the formluas by using **inversion functions** \(r_x(y, z)\) and \(r_y(x, z)\) to recover their values from $z$
4. **Fusion Constraints (for **UNSAT Fusion** only):**
    •	Add additional constraints to enforce logical consistency:
        • \(z = f(x, y)\)
        • \(x = r_x(y, z)\)
        • \(y = r_y(x, z)\)
        
Refer to the following table for predefined fusion and inversion functions categorized by types (Int, Real, String):

#### **Table: Fusion and Inversion Functions**
| Type   | Fusion Function \( f(x, y) \)    | \( r_x(y, z) \)                          | \( r_y(x, z) \)                            |
|--------|----------------------------------|------------------------------------------|--------------------------------------------|
| **Int**  | \( x + y \)                      | \( z - y \)                              | \( z - x \)                                |
|        | \( x + c + y \)                  | \( z - c - y \)                          | \( z - c - x \)                            |
|        | \( x * y \)                 | \( z \, div \, y \)                      | \( z \, div \, x \)                        |
|        | \( c_1 * x + c_2 * y + c_3 \) | \( (z - c_2 * y - c_3) \, div \, c_1 \) | \( (z - c_1 * x - c_3) \, div \, c_2 \) |
| **Real** | \( x + y \)                      | \( z - y \)                              | \( z - x \)                                |
|        | \( x + c + y \)                  | \( z - c - y \)                          | \( z - c - x \)                            |
|        | \( x * y \)                 | \( z / y \)                              | \( z / x \)                                |
|        | \( c_1 * x + c_2 * y + c_3 \) | \( (z - c_2 * y - c_3) / c_1 \)      | \( (z - c_1 * x - c_3) / c_2 \)       |
| **String** | \( x \, str++ \, y \)           | \( str.substr(z, 0, str.len(x)) \)       | \( str.substr(z, str.len(x), str.len(y)) \)|
|        | \( x \, str++ \, c \, str++ \, y \) | \( str.substr(z, 0, str.len(x)) \)       | \( str.replace(str.replace(z, x, ""), c, "") \)|


### SMT-LIB Syntax Instructions:
1.	Variable Declarations: Use declare-fun to declare variables with their types: (declare-fun <var> () <type>).
2.	Function Definitions: Use define-fun to define functions: (define-fun <name> (<arg1 type1> <arg2 type2> ...) <return-type> <body>).
3.	Assertions: Use assert to specify constraints: (assert <constraint>).
4.	Boolean Expressions:
	•	Logical AND: (and <expr1> <expr2> ...)
	•	Logical OR: (or <expr1> <expr2> ...)
	•	Logical NOT: (not <expr>)
5.	Arithmetic Operations: Use standard operators like +, -, *, /, and div for integers.
6.	Relational Operators: Use comparison operators like >, <, >=, <=, and = for numerical expressions.
7.	Equality and Disequality: Use = for equality and distinct for disequality: (distinct <expr1> <expr2> ...).
8.	Conditionals: Use ite for if-then-else expressions: (ite <condition> <then-branch> <else-branch>).
9.	Quantifiers: Use forall and exists for quantified expressions:
	•	Universal: (forall ((x <type>) ...) <body>)
	•	Existential: (exists ((x <type>) ...) <body>)
10.	Constants: Declare constants directly using their types:
	•	Integers: 1, -3
	•	Real numbers: 1.0, -2.5
	•	Booleans: true, false
11.	Parentheses Matching: Ensure all expressions are enclosed in properly matched parentheses.
12.	Comments: Use ; for single-line comments in SMT-LIB.
13. Avoid invalid constructs such as `f(x y) =` or `verify`. SMT-LIB does not support such constructs.

        

## **Output Format:**
Your response **must strictly follow the format below** without additional comments, explanations, or clarifications:
**Fused Problem:**
    [Insert SMT-LIB code for the fused problem here]

•Example 1:
    Problem 1: 
    ( declare-fun x () Real )
    
    ( assert ( not (= (+ (+ 1.0 x ) 6.0) (+ 7.0 x ) ) ) )

    Problem 2:
    ( declare-fun y () Real )
    ( declare-fun w () Real )
    ( declare-fun v () Real )
    
    ( assert ( and ( < y v ) ( >= w v ) ( < (/ w v ) 0) ( > y 0) ) )


    Fused Problem:
    ( declare-fun v () Real )
    ( declare-fun w () Real )
    ( declare-fun x () Real )
    ( declare-fun y () Real )
    ( declare-fun z () Real )
    ( assert ( or
    ( not (= (+ (+ 1.0 (/ z y) ) 6.0) (+ 7.0 x ) ) )
    ( and ( < (/ z x) v ) ( >= w v ) ( < (/ w v ) 0) ( > (/ z x) 0) ) ) )

•Example 2:
    Problem 1:
    ```{formula1}```
    
    Problem 2:
    ```{formula2}```
    
    Fused Problem:
    ?
    [/INST]
    """
    
        ),
        (
            "human",
            f"""
[INST]
Produce the fused problem for these two SMT problems.\
Ensure the fused formula preserves equisatisfiability.\
Only provide the final fused problem syntactically and semantically correct in SMT-LIB format without any additional instructions.\
[/INST]
"""
    ),
    ]
)

In [14]:
llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0,
)

In [8]:

llm = ChatOllama(
    model="Codellama:13b",
    temperature=0,
)

In [15]:
chain = prompt | llm
formula1 = f"""
( declare-fun x () Int )
( declare-fun w () Bool )

( assert (= x (- 1) ) )
( assert (= w (= x (- 1) ) ) )
( assert w )
"""
formula2 = f"""
( declare-fun y () Int )
( declare-fun v () Bool )

( assert (= v ( not (= y (- 1) ) ) ) )
( assert ( ite v false (= y (- 1) ) ) )
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

# Structured output durch keys 

(declare-fun x () Int)
(declare-fun w () Bool)

(assert (= x (- 1) ) )
(assert (= w (= x (- 1) ) ) )
(assert w )

(declare-fun y () Int)
(declare-fun v () Bool)

(assert (= v ( not (= y (- 1) ) ) ) )
(assert (ite v false (= y (- 1) ) ) )

(fuse
  (assert (= (+ (+ 1.0 (/ x y) ) 6.0) (+ 7.0 x ) ))
  (assert (/ x y = z ) )
  (assert (= (/ z y) (/ w v ) ) )
  (assert (= (/ w v ) (/ x y) ) )
)
